In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Inicio la clase con los datos y los pesos iniciales de la red en random

In [6]:
class RNNOnline(nn.Module):
    def __init__(self, input_size=14, hidden_size = 256):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=3, dropout=0.1, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x, hidden):
        # Copia del input para no modificar tensores en el grafo
        x = x.clone()

        #cambio la escala temporal de las variables en nanosegundos
        x[:,:,0] = x[:,:,0] / 1e7
        x[:,:,1] = x[:,:,1] / 1e7

        #normalizo cierto datos del input con los valores que obtuve del robustScale de sckikit-learn. 
        #Elijo estos valores ya que son los que a mayor diferencia de escala se encuentran. En diferencias no tan grandes la propia red aprende la escala sin problemas
        #volumen bid y offer
        x[:,:,4] = (x[:,:,4]-573690) / 515196
        x[:,:,5] = (x[:,:,5]-573690) / 515196

        #price imbalance
        x[:,:,11] = (x[:,:,11]+1.6963328e-06) / 8.28752812e-06

        #depth ratio
        x[:,:,12] = (x[:,:,12]+120097) / 494227

        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :]) 
        return out, hidden

In [4]:
features_cols = ['time_since_last_trade',
                             'time_since_last_update',
                             'spread',
                             'volumen_bid',
                             'volumen_off',
                             'imbalance',
                             'vwap_bid',
                             'vwap_offer',
                             'relative_spread',
                             'log_spread',
                             'price_imbalance',
                             'depth_ratio',
                             'direccion_bid',
                             'direccion_offer']

In [ ]:
Datos = pd.read_csv('Data_MEP')
Datos.pop('mid_price')  #está a otra escala, lo uso para calcular los log_returns pero no para predecir.

X = Datos.iloc[:, :-1].values.astype(np.float32)
y = Datos.iloc[:, -1].values.astype(np.float32)

In [29]:
seq_len = 100

X_seq = []
y_seq = []

for i in range(len(X) - seq_len):
    X_seq.append(X[i:i+seq_len])
    y_seq.append(y[i+seq_len-1])   # target: último paso de la secuencia

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

print(f"Dataset: {X_seq.shape} secuencias, target: {y_seq.shape}")

Dataset: (436133, 100, 14) secuencias, target: (436133,)


In [30]:
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

# Convertir a tensores
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train).unsqueeze(1)  # (N, 1)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test).unsqueeze(1)

In [31]:
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=False)  # online-like
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

In [57]:
model = RNNOnline(hidden_size=64)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
num_epochs = 5
losses_train = []
losses_train_epoch = []
losses_test_epoch = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    hidden = None

    for i, (xb, yb) in enumerate(train_loader):
        optimizer.zero_grad()
        out, hidden = model(xb, hidden)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        hidden = hidden.detach()

        if i % (len(train_loader)//100) == 0:
            pct = 100 * i / len(train_loader)
            losses_train.append(loss.item())
            print(f"Epoch [{epoch+1}/{num_epochs}] - {pct:.1f}% completado")
            print(f"loss_train: {loss}")

        if i % (len(train_loader)//10) == 0:
            torch.save(model.state_dict(), f'checkpoints/Checkpoint_epoch{epoch}_{i}')

    avg_train_loss = total_loss / len(train_loader)
    losses_train_epoch.append(avg_train_loss)

    # VALIDACIÓN
    model.eval()
    with torch.no_grad():
        total_test_loss = 0
        hidden = None
        for xb, yb in test_loader:
            out, hidden = model(xb, hidden)
            loss = criterion(out, yb)
            total_test_loss += loss.item()
            hidden = hidden.detach()

        avg_test_loss = total_test_loss / len(test_loader)
        losses_test_epoch.append(avg_test_loss)

    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.6f} | Test Loss: {avg_test_loss:.6f}")



In [ ]:

plt.plot(losses_train, label="Train")
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.show()

Exporto el modelo entrenado a c++

In [3]:
#Elijo el ultimo modelo
from pathlib import Path

def archivo_mayor_alfabeticamente(carpeta_path):
    carpeta = Path(carpeta_path)
    
    # Filtrar solo archivos y ordenarlos alfabéticamente
    archivos = sorted([f for f in carpeta.iterdir() if f.is_file()])

    if not archivos:
        return None 
    
    # El último archivo en orden alfabético
    return archivos[-1]

# Ejemplo de uso
path = archivo_mayor_alfabeticamente("checkpoints")
print("Archivo mayor alfabéticamente:", path)

Archivo mayor alfabéticamente: checkpoints/Checkpoint_epoch1_0


In [7]:
model = RNNOnline(hidden_size=64)
model.load_state_dict(torch.load(path))
model.eval()

scripted = torch.jit.script(model)
scripted.save("../Codigo_cpp/Modelo_entrenado.pt")